# Multinomial regression

## Reclassification

**Notebook Setup**

In [1]:
import sys
sys.path.append("../src")
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

%matplotlib inline

In [3]:
plt.style.use('ggplot')

In [4]:
%matplotlib inline

from project import rf_models, preprocessing

df = pd.read_csv("../data/re_custody_2017_gsprs.csv", low_memory=False)



# preprocessing
data = preprocessing.preprocess_input_vars_re(df)
data = data[data.ic_custdy_level > 1]
data['high_re_discip_reports'] = np.where(data['re_discip_reports']>2, 1, 0)

data['re_override_up'] = np.where(data['re_ovride_cust_lvl']-data['re_custody_level']>0,1,0)
data['high_re'] = np.where(data['re_custody_level']>3, 1, 0)
print(data.columns)
data = data[
    [
        "gender_female",
        "age_gt_45",
        "age_lt_25",
        "race_B",
        "race_A",
        "race_H",
        "race_I",
        "race_O",
        "off_1_prs_max",
        "off_1_gs_max",
        #"ic_custdy_level",
        "prior_commits",
        "re_discip_reports",
        "re_escp_hist_1",
        #"re_escp_hist_2",
        #"re_escp_hist_3",
        #"re_escp_hist_4",
        "re_escp_hist_5",
        "mrt_stat_DIV",
        "mrt_stat_SEP",
        "mrt_stat_MAR",
        "mrt_stat_WID",
        "employed",
        #"high_re_discip_reports",
        #"high_re"
        "re_custody_level"
    ]
]
data = data.dropna()


df_re_all = data


Index(['re_curr_off_cd_1', 're_curr_off_cd_2', 're_curr_off_cd_3',
       're_prev_off_cd_1', 're_prev_off_cd_2', 're_prev_off_cd_3',
       're_escp_hist_1', 're_escp_hist_2', 're_escp_hist_3', 're_escp_hist_4',
       're_escp_hist_5', 're_discip_reports', 're_age_for_class',
       're_instit_violence', 'ic_prior_commits', 'race', 'sex',
       'ethnic_identity', 'citizenship', 'religion', 'legal_zip_code',
       'ic_employ_ind', 'date_of_birth', 're_custody_level', 'ic_custdy_level',
       'control_number', 're_ovride_cust_lvl', 're_de_year', 'off_1_gs_max',
       'off_1_gs_min', 'off_2_gs_max', 'off_2_gs_min', 'off_3_gs_max',
       'off_3_gs_min', 'off_1_prs_max', 'off_1_prs_min', 'off_2_prs_max',
       'off_2_prs_min', 'off_3_prs_max', 'off_3_prs_min', 'marital_status',
       'ic_mrtl_stat_fr_cl', 'affilatns_ind', 'affilatn_code_1',
       'affilatn_code_2', 'affilatn_code_3', 'affilatn_code_4',
       'affilatn_code_5', 'affilatn_code_6', 'affilatn_code_7',
       'affilat

#### Multinomial

In [5]:
X = df_re_all.drop("re_custody_level", axis=1)
y = df_re_all["re_custody_level"]
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.2, random_state=1)
model = LogisticRegression(multi_class="multinomial", max_iter=10_000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=10000, multi_class='multinomial')

In [6]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X_test, Y_test, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Mean Accuracy: 0.776 (0.009)


In [7]:
model.coef_

array([[ 9.17571936e-01,  6.65397919e-01, -1.87597683e-01,
        -4.66085134e-01, -4.93947435e-01, -1.06172419e-01,
        -3.97606109e-01, -1.14429037e+00, -1.27803624e-01,
        -9.32724597e-02,  4.74383135e-02, -2.16108256e+00,
         8.20398088e-02, -5.21664259e-01,  6.69641995e-01,
         2.94398422e-01,  3.54563233e-01,  5.30459973e-02,
         4.15131848e-01],
       [ 4.06756327e-01,  1.44932886e-01, -1.07105445e-01,
         5.32033431e-02,  8.04874573e-01, -6.07596703e-02,
         6.02962916e-02,  5.55019625e-01,  1.42301290e-02,
        -7.82747497e-03, -2.56832009e-02,  4.78438964e-02,
         2.86325184e-02,  6.27515939e-02, -2.78274361e-01,
         3.46456218e-01,  6.78921180e-02,  4.68967727e-02,
        -1.91932398e-02],
       [-3.97376164e-01, -4.16563427e-01,  3.53125671e-01,
         2.56002018e-01, -2.05578346e-01, -1.29123491e-01,
         5.51887245e-01,  7.03612214e-01,  1.23475490e-01,
         1.02705708e-01, -1.51494442e-02,  1.34106934e+00,
    

In [8]:
np.exp(model.coef_)

array([[2.50320506, 1.94526442, 0.82894814, 0.62745386, 0.61021286,
        0.89926958, 0.67192664, 0.31844982, 0.88002618, 0.91094527,
        1.04858152, 0.11520034, 1.08549902, 0.59353193, 1.95353782,
        1.34231861, 1.42555788, 1.05447815, 1.51457042],
       [1.50193808, 1.15596199, 0.89843093, 1.05464408, 2.23641597,
        0.94104937, 1.06215121, 1.74197517, 1.01433186, 0.99220308,
        0.97464381, 1.04900689, 1.02904637, 1.06476231, 0.75708908,
        1.41404758, 1.07024984, 1.04801382, 0.98098978],
       [0.67208117, 0.65930869, 1.42351003, 1.29175533, 0.8141763 ,
        0.87886543, 1.73652718, 2.02103997, 1.13142227, 1.10816524,
        0.98496473, 3.82312954, 0.78097874, 0.88837966, 0.6235015 ,
        0.63979047, 0.66415698, 0.48772704, 0.9344837 ],
       [0.39575811, 0.67451094, 0.9432513 , 1.16985496, 0.90001055,
        1.34454489, 0.80688233, 0.89195333, 0.99014687, 0.99839552,
        0.9934161 , 2.16445657, 1.14629502, 1.78116695, 1.08441045,
        0.823

In [9]:
# For CL 2
features = list(X_train.columns.values)
coeffs = list(np.exp(model.coef_)[0])
list(zip(features, coeffs))

[('gender_female', 2.5032050625644278),
 ('age_gt_45', 1.9452644240974244),
 ('age_lt_25', 0.8289481401356446),
 ('race_B', 0.6274538639568635),
 ('race_A', 0.6102128579806267),
 ('race_H', 0.8992695831227155),
 ('race_I', 0.6719266410841208),
 ('race_O', 0.3184498182203988),
 ('off_1_prs_max', 0.8800261778185762),
 ('off_1_gs_max', 0.910945270569428),
 ('prior_commits', 1.0485815157590561),
 ('re_discip_reports', 0.11520034211762745),
 ('re_escp_hist_1', 1.0854990214183715),
 ('re_escp_hist_5', 0.5935319348222496),
 ('mrt_stat_DIV', 1.953537819461267),
 ('mrt_stat_SEP', 1.3423186062735821),
 ('mrt_stat_MAR', 1.4255578809871716),
 ('mrt_stat_WID', 1.0544781471772724),
 ('employed', 1.514570420211486)]

In [10]:
# For CL 3
features = list(X_train.columns.values)
coeffs = list(np.exp(model.coef_)[1])
list(zip(features, coeffs))

[('gender_female', 1.5019380795680597),
 ('age_gt_45', 1.1559619860332522),
 ('age_lt_25', 0.8984309329603775),
 ('race_B', 1.054644077964003),
 ('race_A', 2.2364159740681866),
 ('race_H', 0.9410493746978116),
 ('race_I', 1.0621512063876881),
 ('race_O', 1.7419751710271214),
 ('off_1_prs_max', 1.0143318592802906),
 ('off_1_gs_max', 0.9922030799403608),
 ('prior_commits', 0.9746438069453743),
 ('re_discip_reports', 1.0490068887937618),
 ('re_escp_hist_1', 1.029046369358598),
 ('re_escp_hist_5', 1.0647623129405972),
 ('mrt_stat_DIV', 0.7570890776054616),
 ('mrt_stat_SEP', 1.4140475830150097),
 ('mrt_stat_MAR', 1.0702498415193515),
 ('mrt_stat_WID', 1.048013819823259),
 ('employed', 0.980989777701634)]

In [11]:
# For CL 4
features = list(X_train.columns.values)
coeffs = list(np.exp(model.coef_)[2])
list(zip(features, coeffs))

[('gender_female', 0.6720811650978593),
 ('age_gt_45', 0.6593086935591578),
 ('age_lt_25', 1.4235100260774745),
 ('race_B', 1.2917553341585355),
 ('race_A', 0.8141763047079337),
 ('race_H', 0.8788654268398883),
 ('race_I', 1.7365271801800322),
 ('race_O', 2.021039965841173),
 ('off_1_prs_max', 1.1314222732776533),
 ('off_1_gs_max', 1.1081652364668957),
 ('prior_commits', 0.9849647313549769),
 ('re_discip_reports', 3.823129543572266),
 ('re_escp_hist_1', 0.7809787407331134),
 ('re_escp_hist_5', 0.8883796646003911),
 ('mrt_stat_DIV', 0.6235014992189579),
 ('mrt_stat_SEP', 0.6397904692637351),
 ('mrt_stat_MAR', 0.6641569786493356),
 ('mrt_stat_WID', 0.48772703860310485),
 ('employed', 0.9344837018392907)]

In [12]:
# For CL 5
features = list(X_train.columns.values)
coeffs = list(np.exp(model.coef_)[3])
list(zip(features, coeffs))

[('gender_female', 0.39575810567583103),
 ('age_gt_45', 0.6745109445673683),
 ('age_lt_25', 0.9432512989414351),
 ('race_B', 1.1698549579071407),
 ('race_A', 0.9000105514893465),
 ('race_H', 1.3445448856727351),
 ('race_I', 0.8068823344581024),
 ('race_O', 0.8919533324053555),
 ('off_1_prs_max', 0.9901468685370755),
 ('off_1_gs_max', 0.9983955156795937),
 ('prior_commits', 0.9934161011323084),
 ('re_discip_reports', 2.16445657448873),
 ('re_escp_hist_1', 1.1462950236155025),
 ('re_escp_hist_5', 1.7811669546243418),
 ('mrt_stat_DIV', 1.0844104494657278),
 ('mrt_stat_SEP', 0.8234601742039085),
 ('mrt_stat_MAR', 0.9868683754166777),
 ('mrt_stat_WID', 1.8553189217549477),
 ('employed', 0.720235155647592)]

### Feature selection

In [14]:
from sklearn.feature_selection import RFE

data_final_vars=df_re_all.columns.values.tolist()
yvars = ['re_custody_level']
Xvars = [i for i in data_final_vars if i not in yvars]

rfe = RFE(model, n_features_to_select=8, step=1)
rfe = rfe.fit(X, y.values.ravel())

zz= list(zip(Xvars,list(rfe.support_)))
ll = [a for (a,b) in zz if b]
ll

['gender_female',
 'age_gt_45',
 'race_A',
 're_discip_reports',
 'mrt_stat_DIV',
 'mrt_stat_SEP',
 'mrt_stat_MAR',
 'employed']

#### Class imbalance

In [13]:
count_2 = len(df_re_all[df_re_all['re_custody_level']==2])
count_3 = len(df_re_all[df_re_all['re_custody_level']==3])
count_4 = len(df_re_all[df_re_all['re_custody_level']==4])
count_5 = len(df_re_all[df_re_all['re_custody_level']==5])

tot = count_2 + count_3 + count_4 + count_5

pct_2 = count_2/tot
print("percentage of lev 2 is", pct_2*100)
pct_3 = count_3/tot
print("percentage of lev 3 is", pct_3*100)
pct_4 = count_4/tot
print("percentage of lev 4 is", pct_4*100)
pct_5 = count_5/tot
print("percentage of lev 5 is", pct_5*100)


pct_2 +pct_3 + pct_4 +pct_5 

percentage of lev 2 is 60.156464452374415
percentage of lev 3 is 22.36480922316772
percentage of lev 4 is 12.242657150699973
percentage of lev 5 is 5.236069173757891


1.0

#### Oversampling to fix

In [14]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0)

X, y = os.fit_resample(X, y)

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.2, random_state=1)
model = LogisticRegression(multi_class="multinomial", max_iter=10_000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=10000, multi_class='multinomial')

In [18]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X_test, Y_test, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Mean Accuracy: 0.573 (0.007)


In [19]:
np.exp(model.coef_)

array([[5.78411467, 4.37038786, 1.11419576, 1.39118567, 1.26689318,
        2.30928592, 0.69388357, 1.89758085, 0.89575868, 0.90602791,
        1.06800092, 0.12312354, 1.9579385 , 1.84297649, 3.58743955,
        3.59140873, 3.84262969, 1.68862098, 2.8103826 ],
       [1.81427057, 1.49665559, 0.67045756, 0.94500785, 0.80613115,
        0.93140017, 1.9658766 , 1.04320326, 1.04023151, 1.01550026,
        0.9989254 , 1.12237368, 1.02184092, 0.88933676, 0.86831491,
        1.48769178, 0.9359293 , 0.6815824 , 1.30849395],
       [0.35315728, 0.46472802, 1.56162424, 1.16407307, 0.99524094,
        0.90609033, 0.89689664, 0.98302215, 1.06755322, 1.0817903 ,
        0.95019302, 3.50585355, 0.696055  , 0.87882621, 0.59492718,
        0.86932481, 0.65370415, 0.36999303, 0.63079354],
       [0.26983165, 0.32897227, 0.85721678, 0.65343027, 0.9838436 ,
        0.51311473, 0.81736257, 0.51388681, 1.00528553, 1.00469745,
        0.98646909, 2.06408542, 0.71808211, 0.69424212, 0.53960298,
        0.215

In [26]:
# For CL 2
features = list(X_train.columns.values)
coeffs = list(np.exp(model.coef_)[0])
list(zip(features, coeffs))

[('gender_female', 5.784114673411529),
 ('age_gt_45', 4.370387858800114),
 ('age_lt_25', 1.1141957573483274),
 ('race_B', 1.3911856714008854),
 ('race_A', 1.266893179962617),
 ('race_H', 2.309285921361442),
 ('race_I', 0.6938835707509523),
 ('race_O', 1.89758085216362),
 ('off_1_prs_max', 0.8957586832016886),
 ('off_1_gs_max', 0.9060279094585155),
 ('prior_commits', 1.068000918047938),
 ('re_discip_reports', 0.1231235390031406),
 ('re_escp_hist_1', 1.9579384986882713),
 ('re_escp_hist_5', 1.8429764868516205),
 ('mrt_stat_DIV', 3.587439548528901),
 ('mrt_stat_SEP', 3.5914087315800787),
 ('mrt_stat_MAR', 3.8426296913612554),
 ('mrt_stat_WID', 1.6886209791198246),
 ('employed', 2.810382598118759)]

In [27]:
# For CL 3
features = list(X_train.columns.values)
coeffs = list(np.exp(model.coef_)[1])
list(zip(features, coeffs))

[('gender_female', 1.814270568959974),
 ('age_gt_45', 1.4966555859531965),
 ('age_lt_25', 0.6704575632597396),
 ('race_B', 0.9450078502145366),
 ('race_A', 0.8061311517721217),
 ('race_H', 0.9314001666882661),
 ('race_I', 1.9658766034594075),
 ('race_O', 1.0432032576940136),
 ('off_1_prs_max', 1.0402315149253774),
 ('off_1_gs_max', 1.0155002632923493),
 ('prior_commits', 0.9989253995473749),
 ('re_discip_reports', 1.12237367918668),
 ('re_escp_hist_1', 1.0218409214736333),
 ('re_escp_hist_5', 0.8893367599122296),
 ('mrt_stat_DIV', 0.8683149102319874),
 ('mrt_stat_SEP', 1.4876917756764316),
 ('mrt_stat_MAR', 0.9359292990757966),
 ('mrt_stat_WID', 0.6815824030619526),
 ('employed', 1.3084939508902862)]

In [28]:
# For CL 4
features = list(X_train.columns.values)
coeffs = list(np.exp(model.coef_)[2])
list(zip(features, coeffs))

[('gender_female', 0.3531572827629946),
 ('age_gt_45', 0.4647280213862168),
 ('age_lt_25', 1.5616242438223409),
 ('race_B', 1.164073065141239),
 ('race_A', 0.995240937359671),
 ('race_H', 0.9060903314028289),
 ('race_I', 0.8968966375476176),
 ('race_O', 0.9830221465618765),
 ('off_1_prs_max', 1.0675532155358953),
 ('off_1_gs_max', 1.0817903046011734),
 ('prior_commits', 0.95019301948615),
 ('re_discip_reports', 3.5058535500548698),
 ('re_escp_hist_1', 0.6960549957380078),
 ('re_escp_hist_5', 0.878826213604372),
 ('mrt_stat_DIV', 0.5949271827651591),
 ('mrt_stat_SEP', 0.8693248090934237),
 ('mrt_stat_MAR', 0.6537041530285963),
 ('mrt_stat_WID', 0.36999302785121396),
 ('employed', 0.6307935357653568)]

In [29]:
# For CL 5
features = list(X_train.columns.values)
coeffs = list(np.exp(model.coef_)[3])
list(zip(features, coeffs))

[('gender_female', 0.2698316464463032),
 ('age_gt_45', 0.3289722676105704),
 ('age_lt_25', 0.8572167777033057),
 ('race_B', 0.6534302672355999),
 ('race_A', 0.9838436026743212),
 ('race_H', 0.5131147328948413),
 ('race_I', 0.8173625690469667),
 ('race_O', 0.5138868136584845),
 ('off_1_prs_max', 1.0052855340936062),
 ('off_1_gs_max', 1.0046974474051054),
 ('prior_commits', 0.9864690917178893),
 ('re_discip_reports', 2.0640854192854072),
 ('re_escp_hist_1', 0.718082112277222),
 ('re_escp_hist_5', 0.6942421202893079),
 ('mrt_stat_DIV', 0.5396029772151589),
 ('mrt_stat_SEP', 0.21529806660015455),
 ('mrt_stat_MAR', 0.42535072636512355),
 ('mrt_stat_WID', 2.3483126075983405),
 ('employed', 0.43109759395890834)]

In [30]:
rfe = RFE(model, n_features_to_select=4, step=1)
rfe = rfe.fit(X, y.values.ravel())


zz= list(zip(Xvars,list(rfe.support_)))
ll = [a for (a,b) in zz if b]
ll

['gender_female', 'mrt_stat_DIV', 'mrt_stat_SEP', 'mrt_stat_MAR']

## SVM

In [21]:
#We ommitted escape histories 2, 3 and 4 because they were highly correlated with escape history 1
#and because we want to know if they're frequent escapees or not, and not necessarily how many times

from sklearn.metrics import accuracy_score





#applying Support Vector Classifier 
#fitting kernel SVM to training dataset
from sklearn.svm import SVC
classifier_df = SVC(kernel = 'linear' , random_state = 0)
classifier_df.fit(X_train,Y_train)

#predicting test data result
y_pred = classifier_df.predict(X_test)


#setting up accuracy score

acc = accuracy_score(Y_test,y_pred) *100
print("Accuracy for our dataset in predicting test data is : {:.2f}%".format(acc))


Accuracy for our dataset in predicting test data is : 58.39%


In [22]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

         2.0       0.80      0.90      0.85      6992
         3.0       0.55      0.54      0.54      7043
         4.0       0.51      0.58      0.55      7018
         5.0       0.42      0.32      0.36      6999

    accuracy                           0.58     28052
   macro avg       0.57      0.58      0.57     28052
weighted avg       0.57      0.58      0.57     28052



## Neural network

In [23]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', 
                    alpha=1e-5,
                    hidden_layer_sizes=(14,), 
                    random_state=1, 
                    max_iter=10000)

clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)

acc = accuracy_score(Y_test,Y_pred) *100
print("Accuracy for our dataset in predicting test data is : {:.2f}%".format(acc))

Accuracy for our dataset in predicting test data is : 60.47%


In [24]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

         2.0       0.80      0.90      0.85      6992
         3.0       0.55      0.54      0.54      7043
         4.0       0.51      0.58      0.55      7018
         5.0       0.42      0.32      0.36      6999

    accuracy                           0.58     28052
   macro avg       0.57      0.58      0.57     28052
weighted avg       0.57      0.58      0.57     28052



## Random forest

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 0)
clf.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)

acc = accuracy_score(Y_test,y_pred) *100
print("Accuracy for our dataset in predicting test data is : {:.2f}%".format(acc))

Accuracy for our dataset in predicting test data is : 73.13%


In [26]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

         2.0       0.86      0.89      0.87      6992
         3.0       0.64      0.64      0.64      7043
         4.0       0.69      0.69      0.69      7018
         5.0       0.73      0.72      0.72      6999

    accuracy                           0.73     28052
   macro avg       0.73      0.73      0.73     28052
weighted avg       0.73      0.73      0.73     28052

